In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [2]:
# read the last output of lemmatization
LAGT = pd.read_json("../data/large_files/LAGT_v2-0.json")                                                    

In [3]:
LAGT.head()

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,GLAUx?
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνῄσ...",True
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[Θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",True
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ...",False
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...",1982,"[[ἡδύς, τις, ψιθύρισμα, πίτυς, αἰπόλος, ἐκεῖνο...",True
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,152,"[[ῥόδον, δροσόεις, κατάπυκνος, ἐκεῖνος, ἕρπυλλ...",True


In [4]:
LAGT.columns

Index(['filename', 'author', 'title', 'wordcount', 'author_id', 'doc_id',
       'raw_date', 'date_avr', 'date_probs', 'date_manual', 'provenience',
       'tlg_epithet', 'clean_string', 'n_sentences', 'lemmatized_sentences',
       'GLAUx?'],
      dtype='object')

In [26]:
len(LAGT[LAGT["provenience"]=="christian"])

148

In [27]:
# upload manually updated metadata
LAGT_metadata = pd.read_csv("../data/LAGT_metadata-update.csv")
# update dates
# any work cosnidered to be dated between two centuries is dated to the latter
def update_dateavr(date_avr):
    if ".5" not in str(date_avr):
        date_avr += 0.5
    return date_avr
LAGT_metadata["date_avr"] = LAGT_metadata["date_avr"].apply(update_dateavr)

# update provenience
provenience_dict = dict(zip(LAGT_metadata["filename"],  LAGT_metadata["provenience"]))
LAGT["provenience"] = LAGT["filename"].apply(lambda x: provenience_dict[x])

# update author
author_dict = dict(zip(LAGT_metadata["filename"],  LAGT_metadata["author"]))
LAGT["author"] = LAGT["filename"].apply(lambda x: author_dict[x])

# update title
title_dict = dict(zip(LAGT_metadata["filename"],  LAGT_metadata["title"]))
LAGT["title"] = LAGT["filename"].apply(lambda x: title_dict[x])

In [28]:
len(LAGT[LAGT["provenience"]=="christian"])

148

In [9]:
# update dates
# any work cosnidered to be dated between two centuries is dated to the latter
def update_dateavr(date_avr):
    if ".5" not in str(date_avr):
        date_avr += 0.5
    return date_avr
LAGT["date_avr"] = LAGT["date_avr"].apply(update_dateavr)

In [10]:
# non-dated christian texts
LAGT[(LAGT["provenience"]=="christian") & (LAGT["date_avr"].isnull())]

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,GLAUx?
1194,tlg1750.tlg001.1st1K-grc1.xml,Pseudo-Dorotheus,De prophetarum vita et obitu (Dorothei recensio),5347,tlg1750,tlg1750.tlg001,Varia,NaN,{},NaN,christian,[],"Οὗτος Ὠσηὲ τῶν ιβʹ προφητῶν ὁ πρῶτος, ἀξιωθεὶς...",329,"[[ὠσηὲ, προφήτης, πρῶτος, ἀξιωθεὶς, λέγω, δεσπ...",False
1195,tlg1750.tlg002.1st1K-grc1.xml,De prophetarum vita et obitu (recensio anonyma),De prophetarum vita et obitu (recensio anonyma),4110,tlg1750,tlg1750.tlg002,Varia,NaN,{},NaN,christian,[],αʹ. Ἡσαΐας ἀπὸ Ἱερουσαλὴμ θνήσκει ὑπὸ Μανασσῆ ...,329,"[[], [Ἠσαΐας, Ἱεροσόλυμα, θνήσκω, Μανασσῆς, πρ...",False
1196,tlg1750.tlg003.1st1K-grc1.xml,De prophetarum vita et obitu (recensio scholii...,De prophetarum vita et obitu (recensio scholii...,1166,tlg1750,tlg1750.tlg003,Varia,NaN,{},NaN,christian,[],αʹ. Ὠσηὲ ἑρμηνεύεται σωζόμενος. Οὗτος ἦν ἐκ ελ...,105,"[[], [ὠσηὲ, ἑρμηνεύω, σωζόμενος], [ελεμὼθ, φυλ...",False
1197,tlg1750.tlg004.1st1K-grc1.xml,Pseudo-Dorotheus,Index apostolorum discipulorumque Domini,3616,tlg1750,tlg1750.tlg004,Varia,NaN,{},NaN,christian,[],Δωροθέου ἐπισκόπου Τύρου ἀρχαίου ἀνδρὸς πνευμα...,253,"[[δωροθέου, ἐπίσκοπος, Τύρος, ἀρχαῖος, ἀνήρ, π...",False


In [11]:
(LAGT["date_avr"] < 5).sum()

1277

In [15]:
# unfortunately we do not have The Martrdom of Perpetua and Felicita
print(LAGT[LAGT["author_id"]=="tlg2016"]["clean_string"].tolist()[0])

 , , , ; ? , . : , . , , , : : , . , , , ; , : , , . , : , , , . , , . , , , , , : . , . , , , : , , , , , ? : . : , ? : . , , . , : , . , , . : , . : , . ! , . . , . : . , . . : ; : , . : , , . , , : . , . , , : . , , , : , , . , , . , , , . , : , : . : . , , : , . , , , , , : . , : , . , ; , : . , . , : . , : , , : , , ; , ; ; : . : : , . ; : , . , , : , . , ; , : : , . . , : . , . . . . , , : . , : , , : . . : . , ? . : . , , . , : . , : . , , , . . , , : . , , , : ; . , . , . . , , , ; , . , , , . : , . , : . , . . : - : : . . , . , , , ; , : , ; : ; , : . . . . , , , , . , , , , , . , , , . , : , . : , : . , . , . : ; , . . . , . . . , . , : . , , , : , . , : , , ; , , . . . , . : . , . , , . , ; . , . , . , : , . . : , : . : , , . , . , , , , , . , . : : : : . , , , , , . : , : , : . , , : : , ; , ; . : , , . , , ; , . , : , , : . , , ; . ; . : : , . : . , . : . : . : , , . , , , . , : , , . : , ? ? . : . , : ; , . . : ; , . . , . , . , . , , . . , . , ( ) , , ; : . , . . . , : ,

In [16]:
repl_dict = {"μώσης" : "μωυσέως",
                "ἐπισκοπή" : "ἐπίσκοπος",
                #"μαρτύρομαι" : "μαρτυρία",
                #"ἐκκλησιάζω" : "ἐκκλησία",
                # "κοινωνέω" : "κοινωνία"
                }


In [17]:
repl_dict.get("μώσης", "μώσης")

'μωυσέως'

In [18]:
def clean_sentences(sents_list):
    # remove one-letter words:
    sents_list = [[w for w in s if len(w) > 1] for s in sents_list]
    sents_list = [[repl_dict.get(w, w) for w in s] for s in sents_list]
    return sents_list

LAGT["lemmatized_sentences"] = LAGT["lemmatized_sentences"].apply(clean_sentences)

In [19]:
LAGT["lemmatized_sentences"] = LAGT["lemmatized_sentences"].apply(lambda x: [s for s in x if len(s) > 0])

In [20]:
print(LAGT[LAGT["author_id"]=="tlg2016"]["clean_string"].tolist()[0])

 , , , ; ? , . : , . , , , : : , . , , , ; , : , , . , : , , , . , , . , , , , , : . , . , , , : , , , , , ? : . : , ? : . , , . , : , . , , . : , . : , . ! , . . , . : . , . . : ; : , . : , , . , , : . , . , , : . , , , : , , . , , . , , , . , : , : . : . , , : , . , , , , , : . , : , . , ; , : . , . , : . , : , , : , , ; , ; ; : . : : , . ; : , . , , : , . , ; , : : , . . , : . , . . . . , , : . , : , , : . . : . , ? . : . , , . , : . , : . , , , . . , , : . , , , : ; . , . , . . , , , ; , . , , , . : , . , : . , . . : - : : . . , . , , , ; , : , ; : ; , : . . . . , , , , . , , , , , . , , , . , : , . : , : . , . , . : ; , . . . , . . . , . , : . , , , : , . , : , , ; , , . . . , . : . , . , , . , ; . , . , . , : , . . : , : . : , , . , . , , , , , . , . : : : : . , , , , , . : , : , : . , , : : , ; , ; . : , , . , , ; , . , : , , : . , , ; . ; . : : , . : . , . : . : . : , , . , , , . , : , , . : , ? ? . : . , : ; , . . : ; , . . , . , . , . , , . . , . , ( ) , , ; : . , . . . , : ,

In [21]:
LAGT.to_json("../data/large_files/LAGT_v2-1.json")

In [ ]:
LAGT